# This notebook is for the 30 Days of Machine Learning contest on Kaggle.

#### This notebook was created by following Abhishek Thakur's YouTube video tutorial: 
https://www.youtube.com/watch?v=pasJQ-IMm5w&list=PL98nY_tJQXZnP-k3qCDd1hljVSciDV9_N&index=24

In [1]:
# Imports
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression

In [3]:
df = pd.read_csv("data/train_folds.csv")
df_test = pd.read_csv("data/test.csv")
sample_submission = pd.read_csv("data/sample_submission.csv")

df1 = pd.read_csv("train_pred_1.csv")
df1.columns = ["id", "pred_1"]
df2 = pd.read_csv("train_pred_2.csv")
df2.columns = ["id", "pred_2"]
df3 = pd.read_csv("train_pred_3.csv")
df3.columns = ["id", "pred_3"]

df_test1 = pd.read_csv("test_pred_1.csv")
df_test1.columns = ["id", "pred_1"]
df_test2 = pd.read_csv("test_pred_2.csv")
df_test2.columns = ["id", "pred_2"]
df_test3 = pd.read_csv("test_pred_3.csv")
df_test3.columns = ["id", "pred_3"]

df = df.merge(df1, on="id", how="left")
df = df.merge(df2, on="id", how="left")
df = df.merge(df3, on="id", how="left")

df_test = df_test.merge(df_test1, on="id", how="left")
df_test = df_test.merge(df_test2, on="id", how="left")
df_test = df_test.merge(df_test3, on="id", how="left")

In [4]:
df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont9,cont10,cont11,cont12,cont13,target,kfold,pred_1,pred_2,pred_3
0,1,B,B,B,C,B,B,A,E,C,...,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634,0,8.517635,8.530541,8.435535
1,2,B,B,A,A,B,D,A,F,A,...,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233,2,8.385293,8.389950,8.271221
2,3,A,A,A,C,B,D,A,D,A,...,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,4,8.156883,8.163596,8.409983
3,4,B,B,A,C,B,D,A,E,C,...,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253,3,8.356946,8.358030,8.346684
4,6,A,A,A,C,B,D,A,E,A,...,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260,1,8.320270,8.308106,8.284071


In [6]:
useful_features = ["pred_1", "pred_2", "pred_3"]
df_test = df_test[useful_features]

final_test_predictions = []
final_valid_predictions = {}
scores = []
for fold in range(5):
    
    xtrain = df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()
    
    valid_ids = xvalid.id.values.tolist()
    
    
    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]

    model = XGBRegressor(
        random_state=42,
        tree_method='gpu_hist',
        gpu_id=1,
        predictor="gpu_predictor",
        n_estimators=7000,
        learning_rate=0.018728063145787083,
        reg_lambda=0.01636958273385703,
        reg_alpha=2.0005220900285042e-08,
        subsample=0.8948010011350874,
        colsample_bytree=0.12936159520820203,
        max_depth=4,
    )
    model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)
    
print(np.mean(scores), np.std(scores))

final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_1"]
final_valid_predictions.to_csv("level1_train_pred_1.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_1"]
sample_submission.to_csv("level1_test_pred_1.csv", index=False)

[0]	validation_0-rmse:7.54855
[597]	validation_0-rmse:0.71718
0 0.7171552430290576
[0]	validation_0-rmse:7.54527
[560]	validation_0-rmse:0.71712
1 0.7170741021373538
[0]	validation_0-rmse:7.54305
[536]	validation_0-rmse:0.71889
2 0.7188636288803927
[0]	validation_0-rmse:7.54532
[533]	validation_0-rmse:0.71915
3 0.7190854216115767
[0]	validation_0-rmse:7.55077
[548]	validation_0-rmse:0.72207
4 0.7197588325878312
0.7183874456492424 0.0010805385255850382


In [7]:
final_test_predictions = []
final_valid_predictions = {}
scores = []
for fold in range(5):
    
    xtrain = df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()
    
    valid_ids = xvalid.id.values.tolist()
    
    
    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]

    model = RandomForestRegressor(
        n_estimators=500,
        max_depth=3,
        n_jobs=-1,
        random_state=42,
    )
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)
    
print(np.mean(scores), np.std(scores))

final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_2"]
final_valid_predictions.to_csv("level1_train_pred_2.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_2"]
sample_submission.to_csv("level1_test_pred_2.csv", index=False)

0 0.7174499690548006
1 0.7173262536702998
2 0.7191930333851406
3 0.719380858058053
4 0.7178340313837478
0.7182368291104083 0.0008756336261140178


C:\Users\weber\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:41: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


In [8]:
final_test_predictions = []
final_valid_predictions = {}
scores = []
for fold in range(5):
    
    xtrain = df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()
    
    valid_ids = xvalid.id.values.tolist()
    
    
    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]

    model = GradientBoostingRegressor(
    
    )
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)
    
print(np.mean(scores), np.std(scores))

final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_3"]
final_valid_predictions.to_csv("level1_train_pred_3.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_3"]
sample_submission.to_csv("level1_test_pred_3.csv", index=False)

0 0.7172148452703018
1 0.7171596816521136
2 0.7188941600612528
3 0.7192354328126627
4 0.7266739760226507
0.7198356191637963 0.003522464784611569


In [9]:
df = pd.read_csv("data/train_folds.csv")
df_test = pd.read_csv("data/test.csv")
sample_submission = pd.read_csv("data/sample_submission.csv")

df1 = pd.read_csv("level1_train_pred_1.csv")
df1.columns = ["id", "pred_1"]
df2 = pd.read_csv("level1_train_pred_2.csv")
df2.columns = ["id", "pred_2"]
df3 = pd.read_csv("level1_train_pred_3.csv")
df3.columns = ["id", "pred_3"]

df_test1 = pd.read_csv("level1_test_pred_1.csv")
df_test1.columns = ["id", "pred_1"]
df_test2 = pd.read_csv("level1_test_pred_2.csv")
df_test2.columns = ["id", "pred_2"]
df_test3 = pd.read_csv("level1_test_pred_3.csv")
df_test3.columns = ["id", "pred_3"]

df = df.merge(df1, on="id", how="left")
df = df.merge(df2, on="id", how="left")
df = df.merge(df3, on="id", how="left")

df_test = df_test.merge(df_test1, on="id", how="left")
df_test = df_test.merge(df_test2, on="id", how="left")
df_test = df_test.merge(df_test3, on="id", how="left")

df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont9,cont10,cont11,cont12,cont13,target,kfold,pred_1,pred_2,pred_3
0,1,B,B,B,C,B,B,A,E,C,...,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634,0,8.512759,8.529420,8.518059
1,2,B,B,A,A,B,D,A,F,A,...,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233,2,8.405252,8.447558,8.409730
2,3,A,A,A,C,B,D,A,D,A,...,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,4,8.198396,8.153962,8.200279
3,4,B,B,A,C,B,D,A,E,C,...,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253,3,8.363074,8.341485,8.369023
4,6,A,A,A,C,B,D,A,E,A,...,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260,1,8.316082,8.334537,8.316887


In [10]:
useful_features = ["pred_1", "pred_2", "pred_3"]
df_test = df_test[useful_features]

final_test_predictions = []
final_valid_predictions = {}
scores = []
for fold in range(5):
    
    xtrain = df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()
    
    valid_ids = xvalid.id.values.tolist()
    
    
    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]

    model = LinearRegression()
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)
    
print(np.mean(scores), np.std(scores))

0 0.7172384138207639
1 0.7171507223020216
2 0.7189986179097884
3 0.719188747984172
4 0.7193370125843567
0.7183827029202204 0.0009764159606757292


In [13]:
sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.to_csv("submission5.csv", index=False)